# 生成token和训练ML模型

In [1]:
import os,json,sys,logging
sys.path.append("./share")
sys.path.append("./common")
import pandas as pd
import json
from tqdm.notebook import tqdm
from IoTCommon import CIoTCommon
from IoTTotalFeature import CIoTTotalFeature
from IoTSample import CIoTSample
from SHSample import CSHSample
from SHDataProcess import CSHDataProcess
from SHFeatureSelect import CSHFeature
from IoTNoise import CIoTNoise
from Config import g_data_root,get_attack_score
from SHDataEDA import CSHDataDistribution,CSHDataTest
from SHModelClassify import CSHModelClassify
from SHEvaluation import CSHROC
from IoTToken import CIoTToken,CIoTCommonSample
import warnings
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
warnings.simplefilter("ignore")
g_sample_root = "%ssample"%g_data_root
g_text_root = "%stoken"%g_data_root
g_ml_root = "%sml_features"%g_data_root

python(26263) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(26264) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [2]:
ioTSample = CIoTSample()

# 生成token样本

In [3]:
def create_sample_text(ioTSample,attack,num_samples = 0,window_size=30 ):
    group = 0
    df_index = pd.DataFrame()
    sample_list,score_list = CIoTToken.get_sample(ioTSample,attack,num_samples=num_samples,window_size=window_size)
    for i in tqdm(range(len(sample_list)),desc="create text"):
        df_sample = sample_list[i]
        score = score_list[i]
        labels = df_sample['Label'].tolist()
        df_t = df_sample[['segment','id','file','Label']]
        to_drop_columns = []
        to_drop_columns.append('Label')
        to_drop_columns.append('frame.time_utc')
        to_drop_columns.append('eth.dst')
        to_drop_columns.append('eth.src')
        to_drop_columns.append('ip.src_host')
        to_drop_columns.append('ip.dst_host')
        to_drop_columns.append('ip.src')
        to_drop_columns.append('ip.dst')
        to_drop_columns.append('frame.time_delta')
        #to_drop_columns.append('tcp.time_delta')
        #to_drop_columns.append('udp.time_delta')

        df_sample = df_sample.drop(to_drop_columns, axis=1,errors='ignore')
        df_sample = df_sample.dropna(how='all')
        df_sample = df_sample.dropna(axis=1, how='all')
        
        if df_sample.shape[1] < 10:
            continue
        
        df_sample = df_sample.drop(['segment','id','file'], axis=1,errors='ignore')
        df_sample = df_sample.reset_index(drop=True)
        
        df_noised = CIoTNoise.add_df_noise(df_sample,x_columns=df_sample.keys().tolist(),y_columns=['Label'],ratio=0.1)

        text = CIoTCommonSample.get_sample_text(df_sample)
        noised = CIoTCommonSample.get_sample_text(df_noised)

        path = "%s/%s"%(g_text_root,attack)
        os.makedirs("%s/ml/attack"%(path), exist_ok=True)
        os.makedirs("%s/ml/noised"%(path), exist_ok=True)
        os.makedirs("%s/llm/attack"%(path), exist_ok=True)
        os.makedirs("%s/llm/noised"%(path), exist_ok=True)

        ml_token_file = "%s/ml/attack/G%d-%f.csv"%(path,group,score)
        ml_noised_file = "%s/ml/noised/G%d-%f.csv"%(path,group,score)
        llm_token_file = "%s/llm/attack/G%d-%f.txt"%(path,group,score)
        llm_noised_file = "%s/llm/noised/G%d-%f.txt"%(path,group,score)
        df_t['attack'] = attack
        df_t['sample_file'] = df_t['file']
        df_t['llm_token_file'] = llm_token_file
        df_t['llm_noised_file'] = llm_noised_file
        df_t['ml_token_file'] = ml_token_file
        df_t['ml_noised_file'] = ml_noised_file
        df_t['score'] = score
        df_t['group'] = group
        df_t['Label'] = json.dumps(labels)
        df_t['id'] = df_t['id'].astype(int)
        df_index = pd.concat([df_index,df_t],ignore_index=True)
        
        df_sample.to_csv(ml_token_file,index=False)
        df_noised.to_csv(ml_noised_file,index=False)
        
        with open(llm_token_file,"w") as fp:
            fp.write(text)
        with open(llm_noised_file,"w") as fp:
            fp.write(noised)
            
        group = group + 1
        
    del df_t['file']
    df_index = df_index.reset_index(drop = True)
    df_index.to_csv("%s/%s/index.csv"%(g_text_root,attack))
    display(df_index['id'].nunique(),df_index.shape[0])

In [ ]:
for attack in tqdm(ioTSample.get_attack_type(),desc="Total"):
    create_sample_text(ioTSample,attack,10000)

python(26265) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Total:   0%|          | 0/14 [00:00<?, ?it/s]

Port Scanning attack:   0%|          | 0/3 [00:00<?, ?it/s]

create text:   0%|          | 0/22424 [00:00<?, ?it/s]

9955

...

create text:   0%|          | 0/157561 [00:00<?, ?it/s]

6249

609300

Password attacks:   0%|          | 0/103 [00:00<?, ?it/s]

create text:   0%|          | 0/1002879 [00:00<?, ?it/s]

# 训练模型

In [4]:
ioTFeature = CIoTTotalFeature()
all_data = []
g_feature_type = {}
for protocol in ioTFeature.get_protocols():
    for feature in ioTFeature.get_features(protocol):
        type = ioTSample.get_type(feature)
        all_data.append({"feature":feature,"type":type})
        g_feature_type[feature] = type
df_feature_type = pd.DataFrame(all_data)

In [5]:
def create_sample(ioTSample,attack,num_samples = 5000, window_size=30 ):
    sample_list,score_list = CIoTToken.get_sample(ioTSample,attack,num_samples=num_samples,window_size=window_size)
    df_data = pd.DataFrame()
    all_data = []
    for i in tqdm(range(len(sample_list)),desc="Create sample"):
        df_sample = sample_list[i]
        score = score_list[i]
        df_sample = df_sample.drop(['segment','id','file'], axis=1,errors='ignore')
        df_sample = df_sample.dropna(how='all')
        df_sample = df_sample.dropna(axis=1, how='all')
        if df_sample.shape[1] < 10:
            continue
        df_sample['frame.time_utc']=df_sample['frame.time_utc'].fillna(method='ffill')
        df_sample['frame.time_utc']=pd.to_datetime(df_sample['frame.time_utc'],format='%Y-%m-%d %H:%M:%S.%f')
        df_sample['frame.time_utc']=df_sample['frame.time_utc'].fillna(method='ffill')
        df_sample['frame.time_utc']=df_sample['frame.time_utc'].apply(lambda x: x.timestamp() if pd.notnull(x) else None)

        df_sample = ioTSample.format(df_sample)
        for key,type in zip(df_sample.dtypes.index,df_sample.dtypes):
            if type in ['object','datetime64[ns]']:
                del df_sample[key]
        
        df_sample, scale_columns = CSHDataProcess.get_scale(df_sample,y_column='Label')
        df_sample['score'] = score
        df_sample['group'] = i
        all_data.extend(json.loads(df_sample.to_json(orient='records')))
        #if len(all_data) > 100:
        #    break
        #df_data = pd.concat([df_data, df_sample], ignore_index=True)
                        
    df_data = pd.DataFrame(all_data)
    df_data = df_data.reset_index(drop=True)
    
    all_data = []
    for item,df_tmp in tqdm(df_data.groupby(['group','score']),desc="flatten sample"):
        group = item[0]
        score = item[1]
        labels = df_tmp['Label'].tolist()
        del df_tmp['group']
        del df_tmp['score']
        del df_tmp['Label']
        df_one_row = pd.DataFrame([df_tmp.values.flatten()], columns=[f"{col}_{i}" for i in range(len(df_tmp)) for col in df_tmp.columns])
        df_one_row['score'] = score
        df_one_row['Label'] = json.dumps(labels)
        all_data.extend(json.loads(df_one_row.to_json(orient='records')))
        #df_result = pd.concat([df_result, df_one_row], ignore_index=True)

    df_result = pd.DataFrame(all_data)
    df_result = df_result.reset_index(drop=True)
    df_result = df_result.fillna(0)
    return df_result

In [ ]:
flag = False
for attack in tqdm(ioTSample.get_attack_type(),desc="Total"):
    #if attack == 'Vulnerability scanner attack':
    #    flag = True
    #if not flag:
    #    continue
    df_data = create_sample(ioTSample,attack,num_samples=20000)
    file_name = "%s/%s.csv"%(g_ml_root,attack)
    os.makedirs(g_ml_root, exist_ok=True)
    df_data.to_csv(file_name,index=False)

flatten sample:   0%|          | 0/157532 [00:00<?, ?it/s]

# 查看计算的分值与label的关系

In [ ]:
df = pd.read_csv("%s/Port Scanning attack.csv"%g_ml_root,low_memory=False)
df_data = df[['score','Label']]
all_data = []
for i,row in df_data.iterrows():
    tmp = {}
    tmp['score'] = row['score']
    labels = json.loads(row['Label'])
    tmp['sum'] = 0
    for i in range(len(labels)):
        tmp["L_%d"%i] = labels[i]
        tmp['sum'] = tmp['sum'] + labels[i]
    all_data.append(tmp)
df_data = pd.DataFrame(all_data)

In [ ]:
df_tmp = df_data[['score','sum']]
df_tmp.hist(bins=100)
df_tmp['score'] = (df_tmp['score'] - df_tmp['score'].mean())/(df_tmp['score'].max() - df_tmp['score'].min())
df_tmp['sum'] = (df_tmp['sum'] - df_tmp['sum'].mean())/(df_tmp['sum'].max() - df_tmp['score'].min())
df_tmp.tail(1000).reset_index(drop=True).plot()